In [1]:
import requests
import json
import re
import sys
from bs4 import BeautifulSoup
import pandas as pd

soup = BeautifulSoup('html.parser')
soup

<html><body><p>html.parser</p></body></html>

In [2]:
# API 데이터 정보 정리

real_estate_schema = {
    "si": '시도',
    "deal_ymd": '년월',
    "sggNm": '구',
    "sggCd": "법정동시군구코드",
    "umdCd": "법정동읍면동코드",
    "landCd": "법정동지번코드",
    "bonbun": "법정동본번코드",
    "bubun": "법정동부번코드",
    "roadNm": "도로명",
    "roadNmSggCd": "도로명시군구코드",
    "roadNmCd": "도로명코드",
    "roadNmSeq": "도로명일련번호코드",
    "roadNmbCd": "도로명지상지하코드",
    "roadNmBonbun": "도로명건물본번호코드",
    "roadNmBubun": "도로명건물부번호코드",
    "umdNm": "법정동",
    "aptNm": "아파트명",
    "jibun": "지번",
    "excluUseAr": "전용면적",
    "dealYear": "계약년도",
    "dealMonth": "계약월",
    "dealDay": "계약일",
    "dealAmount": "거래금액(만원)",
    "floor": "층",
    "buildYear": "건축년도",
    "aptSeq": "단지 일련번호",
    "cdealType": "해제여부",
    "cdealDay": "해제사유발생일",
    "dealingGbn": "거래유형(중개 및 직거래 여부)",
    "estateAgentSggNm": "중개사소재지(시군구 단위)",
    "rgstDate": "등기일자",
    "aptDong": "아파트 동명",
    "slerGbn": "거래주체정보_매도자(개인/법인/공공기관/기타)",
    "buyerGbn": "거래주체정보_매수자(개인/법인/공공기관/기타)",
    "landLeaseholdGbn": "토지임대부 아파트 여부"
}

# 서울시 25개 구의 법정동 코드 앞 5자리
SEOUL_DISTRICTS = {
    "강남구": "11680",
    "강동구": "11740",
    "강북구": "11305",
    "강서구": "11500",
    "관악구": "11620",
    "광진구": "11215",
    "구로구": "11530",
    "금천구": "11545",
    "노원구": "11350",
    "도봉구": "11320",
    "동대문구": "11230",
    "동작구": "11590",
    "마포구": "11440",
    "서대문구": "11410",
    "서초구": "11650",
    "성동구": "11200",
    "성북구": "11290",
    "송파구": "11710",
    "양천구": "11470",
    "영등포구": "11560",
    "용산구": "11170",
    "은평구": "11380",
    "종로구": "11110",
    "중구": "11140",
    "중랑구": "11260"
}

GYEONGGI_DISTRICTS = {
    "가평군": "41820",
    "고양시 덕양구": "41281",
    "고양시 일산동구": "41285",
    "고양시 일산서구": "41287",
    "과천시": "41290",
    "광명시": "41210",
    "광주시": "41610",
    "구리시": "41310",
    "군포시": "41410",
    "김포시": "41570",
    "남양주시": "41360",
    "동두천시": "41250",
    "부천시": "41190",
    "성남시 분당구": "41135",
    "성남시 수정구": "41131",
    "성남시 중원구": "41133",
    "수원시 권선구": "41115",
    "수원시 영통구": "41117",
    "수원시 장안구": "41111",
    "수원시 팔달구": "41113",
    "시흥시": "41390",
    "안산시 단원구": "41273",
    "안산시 상록구": "41271",
    "안성시": "41550",
    "안양시 동안구": "41173",
    "안양시 만안구": "41171",
    "양주시": "41630",
    "양평군": "41830",
    "여주시": "41670",
    "연천군": "41800",
    "오산시": "41370",
    "용인시 기흥구": "41463",
    "용인시 수지구": "41465",
    "용인시 처인구": "41461",
    "의왕시": "41430",
    "의정부시": "41150",
    "이천시": "41500",
    "파주시": "41480",
    "평택시": "41220",
    "포천시": "41650",
    "하남시": "41450",
    "화성시": "41590"
}

# 인천광역시 10개 구/군 법정동 코드 (앞 5자리)
INCHEON_DISTRICTS = {
    "강화군": "28710",
    "계양구": "28245",
    "남동구": "28200",
    "동구": "28140",
    "미추홀구": "28177",  # (구)남구
    "부평구": "28237",
    "서구": "28260",
    "연수구": "28185",
    "옹진군": "28720",
    "중구": "28110"
 }


In [21]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import xml.etree.ElementTree as ET

# ✅ API 기본 설정
BASE_URL = "http://apis.data.go.kr/1613000/RTMSDataSvcAptTradeDev/getRTMSDataSvcAptTradeDev"
SERVICE_KEY = "YPxXvm/3jO5hggkbdJFWnhh8IC3VdMlkRvZHh1pkG8eZxXw12ymm4OAW10urfUVN++TzkurcrZ5Os3Gr9P8Kwg=="

# 크롤링할 지역 설정
# regions = ['서울', '경기', '인천']
regions = ['인천']

# ✅ 크롤링할 기간 설정
START_YEAR = 2025
START_MONTH = 1
END_YEAR = 2025
END_MONTH = 2

# ✅ 결과 저장할 DataFrame
df_real_estate_data = pd.DataFrame()

# ✅ 데이터 수집 시작
for region in regions:
    for year in range(START_YEAR, END_YEAR + 1):
        for month in range(1, 13):
            if year == END_YEAR and month > END_MONTH:
                break

            deal_ymd = f"{year}{month:02}"  # 거래 연월 (YYYYMM 형식)
            print(f"⏳ {deal_ymd} 데이터 수집 시작")
        
            gu_list = None
            if region == '서울':
                gu_list = SEOUL_DISTRICTS.items()
            if region == '경기':
                gu_list = GYEONGGI_DISTRICTS.items()
            if region == '인천':
                gu_list = INCHEON_DISTRICTS.items()

            # 각 구별 데이터 요청
            for gu, code in gu_list:
                print(f"📌 {gu} ({code}) 데이터 요청 중...")

                params = {
                    "LAWD_CD": code,
                    "DEAL_YMD": deal_ymd,  # 거래 연월
                    "serviceKey": SERVICE_KEY,
                    "pageNo": 1,
                    "numOfRows": 999
                }

                try:
                    response = requests.get(BASE_URL, params=params)
                    response.encoding = "utf-8"

                    # 응답 상태 코드 체크
                    if response.status_code != 200:
                        print(f"❌ {gu} 요청 실패 (Status Code: {response.status_code})")
                        continue

                    # XML 파싱
                    soup = BeautifulSoup(response.text, "xml")
                    items = soup.find_all("item")

                    if not items:
                        print(f"⚠️ {gu}에 거래 데이터가 없습니다.")
                        continue

                    # ✅ 데이터프레임 변환
                    for item in items:
                        xml_data = str(item)
                        try:
                            root = ET.fromstring(xml_data)
                            data = {
                                'si': region,
                                'deal_ymd': deal_ymd,  # 거래 연월
                                'sggNm': gu,  # 구 이름
                                'aptDong': root.find('aptDong').text if root.find('aptDong') is not None else '',
                                'aptNm': root.find('aptNm').text if root.find('aptNm') is not None else '',
                                'aptSeq': root.find('aptSeq').text if root.find('aptSeq') is not None else '',
                                'bonbun': root.find('bonbun').text if root.find('bonbun') is not None else '',
                                'bubun': root.find('bubun').text if root.find('bubun') is not None else '',
                                'buildYear': root.find('buildYear').text if root.find('buildYear') is not None else '',
                                'buyerGbn': root.find('buyerGbn').text if root.find('buyerGbn') is not None else '',
                                'cdealDay': root.find('cdealDay').text if root.find('cdealDay') is not None else '',
                                'dealAmount': root.find('dealAmount').text if root.find('dealAmount') is not None else '',
                                'dealDay': root.find('dealDay').text if root.find('dealDay') is not None else '',
                                'dealMonth': root.find('dealMonth').text if root.find('dealMonth') is not None else '',
                                'dealYear': root.find('dealYear').text if root.find('dealYear') is not None else '',
                                'excluUseAr': root.find('excluUseAr').text if root.find('excluUseAr') is not None else '',
                                'floor': root.find('floor').text if root.find('floor') is not None else '',
                                'jibun': root.find('jibun').text if root.find('jibun') is not None else '',
                                'roadNm': root.find('roadNm').text if root.find('roadNm') is not None else '',
                                'roadNmBonbun': root.find('roadNmBonbun').text if root.find('roadNmBonbun') is not None else '',
                                'roadNmBubun': root.find('roadNmBubun').text if root.find('roadNmBubun') is not None else '',
                                'roadNmCd': root.find('roadNmCd').text if root.find('roadNmCd') is not None else '',
                                'roadNmSeq': root.find('roadNmSeq').text if root.find('roadNmSeq') is not None else '',
                                'roadNmSggCd': root.find('roadNmSggCd').text if root.find('roadNmSggCd') is not None else '',
                                'sggCd': root.find('sggCd').text if root.find('sggCd') is not None else '',
                                'umdCd': root.find('umdCd').text if root.find('umdCd') is not None else '',
                                'umdNm': root.find('umdNm').text if root.find('umdNm') is not None else ''
                            }
                            df = pd.DataFrame([data])
                            df_real_estate_data = pd.concat([df_real_estate_data, df], ignore_index=True)

                        except ET.ParseError:
                            print(f"❌ XML 파싱 오류 발생: {xml_data[:200]}")  # 일부 데이터만 출력
                            continue

                    print(f"✅ {gu} 데이터 수집 완료 ({len(items)}건)")

                except requests.RequestException as e:
                    print(f"❌ API 요청 중 오류 발생: {e}")
                
                time.sleep(0.5)  # API 요청 간격 조절

print("📦 데이터 수집 완료")

# 칼럼명 변환
df_real_estate_data.rename(columns=real_estate_schema, inplace=True)

# ✅ 결과 확인
print(f"📊 최종 데이터프레임 크기: {df_real_estate_data.shape}")


⏳ 202501 데이터 수집 시작
📌 강화군 (28710) 데이터 요청 중...
✅ 강화군 데이터 수집 완료 (8건)
📌 계양구 (28245) 데이터 요청 중...
✅ 계양구 데이터 수집 완료 (142건)
📌 남동구 (28200) 데이터 요청 중...
✅ 남동구 데이터 수집 완료 (251건)
📌 동구 (28140) 데이터 요청 중...
✅ 동구 데이터 수집 완료 (41건)
📌 미추홀구 (28177) 데이터 요청 중...
✅ 미추홀구 데이터 수집 완료 (167건)
📌 부평구 (28237) 데이터 요청 중...
✅ 부평구 데이터 수집 완료 (214건)
📌 서구 (28260) 데이터 요청 중...
✅ 서구 데이터 수집 완료 (345건)
📌 연수구 (28185) 데이터 요청 중...
✅ 연수구 데이터 수집 완료 (240건)
📌 옹진군 (28720) 데이터 요청 중...
⚠️ 옹진군에 거래 데이터가 없습니다.
📌 중구 (28110) 데이터 요청 중...
✅ 중구 데이터 수집 완료 (93건)
⏳ 202502 데이터 수집 시작
📌 강화군 (28710) 데이터 요청 중...
✅ 강화군 데이터 수집 완료 (14건)
📌 계양구 (28245) 데이터 요청 중...
✅ 계양구 데이터 수집 완료 (201건)
📌 남동구 (28200) 데이터 요청 중...
✅ 남동구 데이터 수집 완료 (353건)
📌 동구 (28140) 데이터 요청 중...
✅ 동구 데이터 수집 완료 (57건)
📌 미추홀구 (28177) 데이터 요청 중...
✅ 미추홀구 데이터 수집 완료 (239건)
📌 부평구 (28237) 데이터 요청 중...
✅ 부평구 데이터 수집 완료 (322건)
📌 서구 (28260) 데이터 요청 중...
✅ 서구 데이터 수집 완료 (400건)
📌 연수구 (28185) 데이터 요청 중...
✅ 연수구 데이터 수집 완료 (370건)
📌 옹진군 (28720) 데이터 요청 중...
⚠️ 옹진군에 거래 데이터가 없습니다.
📌 중구 (28110) 데이터 요청 중...
✅ 중구 데이터 수집 완료 (123건

In [22]:
# 변환 후 저장

df_real_estate_data.to_csv('인천_2025_월별_법정동별_실거래가.csv', index=False, encoding='cp949')

df_test = pd.read_csv('인천_2025_월별_법정동별_실거래가.csv', encoding='cp949')

df_test['거래금액(만원)'] = df_test['거래금액(만원)'].str.replace(",", "").astype(int)
df_test['전용면적당 거래금액(만원)'] = df_test['거래금액(만원)'] / df_test['전용면적']

df_test = df_test.groupby(
    ['년월', '구', '법정동', '법정동시군구코드', '법정동읍면동코드'], as_index=False
).agg({'거래금액(만원)': 'mean', '전용면적당 거래금액(만원)': 'mean'})

df_test.rename(
  columns={
    '거래금액(만원)': '법정동 평균 거래금액(만원)', 
    '전용면적당 거래금액(만원)': '법정동 평균 전용면적당 거래금액(만원)'
  }
)

def apply_bcode(row):
    bcode = int(str(row['법정동시군구코드']) + str(row['법정동읍면동코드']))
    return bcode

df_test['법정동코드'] = df_test.apply(apply_bcode, axis=1)

df_test.to_csv('인천_2025_월별_법정동별_실거래가_평균.csv', index=False, encoding='cp949')

In [23]:
df_test = pd.read_csv('인천_2025_월별_법정동별_실거래가_평균.csv', encoding='cp949')
df_test

,년월,구,법정동,법정동시군구코드,법정동읍면동코드,거래금액(만원),전용면적당 거래금액(만원),법정동코드
0,202501,강화군,강화읍 갑곳리,28710,25025,7400.000000,141.275296,2871025025
1,202501,강화군,강화읍 남산리,28710,25024,37000.000000,358.217163,2871025024
2,202501,강화군,선원면 창리,28710,31025,26009.666667,374.132577,2871031025
3,202501,계양구,계산동,28245,10200,28763.333333,441.189069,2824510200
4,202501,계양구,귤현동,28245,11100,46060.000000,478.134002,2824511100
...,...,...,...,...,...,...,...,...
149,202502,중구,신흥동3가,28110,12800,24562.500000,334.199744,2811012800
150,202502,중구,운남동,28110,14600,41178.947368,518.146778,2811014600
151,202502,중구,운서동,28110,14700,34736.363636,420.077900,2811014700
152,202502,중구,중산동,28110,14500,37164.912281,501.005397,2811014500
